# ML Assortment Optimization pipeline

This is the complete **Machine Learning (Random Forest) Pipeline** for the project.

### **The Strategy**

1.  **Data Preparation (The "Training Set")**:

      * We transform the raw transaction logs into a **binary classification dataset**.
      * Each row represents a single **User-Item pair** from the history.
      * **Features:** Same as MNL (Cuisine Match, Price Gap, Rating, ETA).
      * **Target ($Y$):** `1` if the user bought this item, `0` if they didn't.
      * *Crucial Detail:* Since the original offer sets had 5 items and the user picked 1, our training data will have roughly four `0`s for every `1` (imbalanced data).

2.  **Model Training (Random Forest)**:

      * We train a `RandomForestClassifier` to predict $P(\text{purchase} \mid \text{user}, \text{item})$.
      * Unlike MNL, which learns a single vector $\beta$, the Random Forest can learn non-linear interactions (e.g., "Users only care about rating if the price is low").

3.  **Personalized Optimization (Heuristic)**:

      * To find the best menu for a *new* user, we don't use an LP or greedy revenue formula.
      * Instead, we use the **"Independent Ranking" Heuristic** (standard in industry ML):
        1.  Score *every* restaurant in the universe (100 items) for this user.
        2.  Calculate **Expected Revenue Score**: $\text{Score}_i = \hat{P}(\text{buy}_i) \times \text{Price}_i$.
        3.  Sort and pick the top $K=5$.

4.  **Ground-Truth Evaluation (The Oracle)**:

      * We use the exact same "God View" function as before to see what the user *actually* buys from this ML-generated menu.

In [ ]:
# =============================================================================
# IEOR 145: END-TO-END ML (RANDOM FOREST) PIPELINE
# =============================================================================

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from ast import literal_eval

# =============================================================================
# 1. SETUP & DATA LOADING
# =============================================================================
print("--- 1. Loading Data ---")

tx_df = pd.read_csv("groundtruth_transaction_data.csv")
restaurants = pd.read_csv("berkeley_real_restaurants_100.csv")

# Extract Users (ensuring we get profile_name this time!)
user_cols = ['user_id', 'x', 'y', 'profile', 'profile_name', 'price_tolerance', 'cuisine_rank']
users = tx_df[user_cols].drop_duplicates(subset='user_id').reset_index(drop=True)

# Helper Parsers
def parse_cuisine_rank(val):
    if isinstance(val, dict): return val
    if not isinstance(val, str): return {}
    s = val.replace("np.str_(", "").replace(")", "")
    try: return literal_eval(s)
    except: return {}

def parse_offer_set(val):
    if isinstance(val, list): return val
    if isinstance(val, str):
        try: return literal_eval(val)
        except: return []
    return []

users['cuisine_rank_dict'] = users['cuisine_rank'].apply(parse_cuisine_rank)
tx_df['offer_set_ids'] = tx_df['offer_set_ids'].apply(parse_offer_set)

print(f"Loaded {len(restaurants)} restaurants.")
print(f"Loaded {len(tx_df)} transactions.")

# =============================================================================
# 2. FEATURE ENGINEERING (ML FORMAT)
# =============================================================================
print("\n--- 2. Preparing ML Training Data ---")

def get_ml_features(user_row, rest_row):
    """
    Returns feature vector for ML Model.
    Note: We separate Price and Tolerance to let RF learn the interaction,
    or we can keep the engineered 'gap'. Let's use the same strong features as MNL.
    """
    # 1. Cuisine Match
    ranks = user_row['cuisine_rank_dict']
    C = len(ranks) if len(ranks) > 0 else 1
    r_rank = ranks.get(rest_row['cuisine'], C)
    x_cuisine = (C - r_rank) / (C - 1) if C > 1 else 0.0

    # 2. Price Gap (Price - Tolerance)
    x_price_gap = float(rest_row['price_level']) - float(user_row['price_tolerance'])

    # 3. Rating
    x_rating = (float(rest_row['rating_5star']) - 3.0) / 2.0

    # 4. ETA
    dx = user_row['x'] - rest_row['x']
    dy = user_row['y'] - rest_row['y']
    x_eta = np.sqrt(dx**2 + dy**2) / 10.0

    # 5. Raw Price (Optional, but good for RF to see absolute price)
    x_price_raw = float(rest_row['price_level'])

    return [x_cuisine, x_price_gap, x_rating, x_eta, x_price_raw]

feature_names = ["CuisineMatch", "PriceGap", "Rating", "ETA", "PriceRaw"]

# Build Classification Dataset
# Each row = (User, Item) -> Label (1 if chosen, 0 if not)
ml_rows = []

# Use a sample for speed (e.g. 2000 txns)
train_tx = tx_df.sample(n=min(2000, len(tx_df)), random_state=42)

for idx, row in train_tx.iterrows():
    u_id = row['user_id']
    user = users[users['user_id'] == u_id].iloc[0]
    offer_ids = row['offer_set_ids']
    chosen_id = row['chosen_id']

    for rid in offer_ids:
        rest = restaurants[restaurants['restaurant_id'] == rid].iloc[0]
        feats = get_ml_features(user, rest)
        label = 1 if rid == chosen_id else 0

        ml_rows.append(feats + [label])

# Convert to DataFrame
train_df = pd.DataFrame(ml_rows, columns=feature_names + ["Label"])

X_train = train_df[feature_names]
y_train = train_df["Label"]

print(f"Training Data: {len(train_df)} rows.")
print(f"Class Balance: {y_train.mean():.1%} positive (chosen).")

# =============================================================================
# 3. TRAIN RANDOM FOREST
# =============================================================================
print("\n--- 3. Training Random Forest ---")

rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=5,        # Keep simple to avoid overfitting on small data
    class_weight="balanced", # Handle the 1-vs-4 imbalance
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train, y_train)

# Feature Importance Check
importances = dict(zip(feature_names, rf_model.feature_importances_))
print("Feature Importances:")
for f, imp in sorted(importances.items(), key=lambda x: x[1], reverse=True):
    print(f"  {f:12s}: {imp:.4f}")

# =============================================================================
# 4. PERSONALIZED OPTIMIZATION (SCORING)
# =============================================================================
print("\n--- 4. Defining ML Optimizer ---")

def get_ml_optimal_assortment(user, rf_model, K=5):
    """
    1. Featurize ALL 100 restaurants for this user.
    2. Predict P(buy) for each.
    3. Score = P(buy) * Price.
    4. Pick Top K.
    """
    # Bulk create features for all restaurants
    # (Much faster than looping row-by-row)
    user_features_list = []
    prices = []
    ids = []

    for _, rest in restaurants.iterrows():
        feats = get_ml_features(user, rest)
        user_features_list.append(feats)
        prices.append(rest['price_level'])
        ids.append(rest['restaurant_id'])

    X_score = pd.DataFrame(user_features_list, columns=feature_names)

    # Predict Probabilities (Class 1)
    probs = rf_model.predict_proba(X_score)[:, 1]

    # Calculate Expected Revenue Score
    scores = probs * np.array(prices)

    # Get Top K Indices
    # argsort sorts ascending, so we take negative to sort descending
    top_k_indices = np.argsort(-scores)[:K]

    return [ids[i] for i in top_k_indices]

# =============================================================================
# 5. GROUND TRUTH EVALUATION (ORACLE)
# =============================================================================
print("\n--- 5. Defining Ground Truth (Oracle) ---")

# (Same Profile Definitions as MNL)
PROFILES = {
    1: [0.0, 2.5, 0.7, 0.8, 0.7], 2: [0.0, 1.0, 2.2, 0.4, 0.6],
    3: [0.0, 1.0, 0.8, 2.8, 0.5], 4: [0.0, 0.9, 0.7, 0.7, 2.5],
    5: [0.0, 1.3, 1.0, 2.1, 0.7], 6: [0.0, 1.5, 1.5, 1.5, 1.5],
    7: [0.0, 1.8, 0.3, 1.2, 0.9], 8: [0.0, 1.2, 1.9, 0.9, 0.3],
    9: [0.0, 1.0, 1.0, 0.8, 1.8], 10:[0.0, 1.2, 0.8, 2.0, 1.7]
}

def get_gt_utility(user, rest):
    """Calculates True Utility (Same as MNL Pipeline)"""
    ranks = user['cuisine_rank_dict']
    C = len(ranks) if len(ranks) > 0 else 1
    r_rank = ranks.get(rest['cuisine'], C)
    cm = (C - r_rank) / (C - 1) if C > 1 else 0.0

    diff = user['price_tolerance'] - rest['price_level']
    pp = abs(diff) * (0.5 if diff >= 0 else 1.5)

    rn = (rest['rating_5star'] - 3.0) / 2.0
    dx, dy = user['x'] - rest['x'], user['y'] - rest['y']
    eta = np.sqrt(dx**2 + dy**2) / 10.0

    beta = PROFILES[user['profile']]
    # U = b0 + b1*CM - b2*PP + b3*RN - b4*ETA
    return beta[0] + (beta[1]*cm) - (beta[2]*pp) + (beta[3]*rn) - (beta[4]*eta)

def evaluate_ground_truth(user, assortment_ids):
    best_util = -np.inf
    revenue = 0.0
    chosen_id = None

    for rid in assortment_ids:
        rest = restaurants[restaurants['restaurant_id'] == rid].iloc[0]
        u_true = get_gt_utility(user, rest)

        if u_true > best_util:
            best_util = u_true
            revenue = rest['price_level']
            chosen_id = rid

    return revenue, chosen_id

# =============================================================================
# 6. RUNNING EVALUATION & REPORTING
# =============================================================================
print("\n--- 6. Running ML Pipeline Evaluation (N=200) ---")

rest_name_map = restaurants.set_index('restaurant_id')['name'].to_dict()
results = []
test_users = users.head(200)

for _, user in test_users.iterrows():
    # 1. Optimize (ML)
    rec_ids = get_ml_optimal_assortment(user, rf_model, K=5)
    rec_names = [rest_name_map[rid] for rid in rec_ids]

    # 2. Evaluate (Oracle)
    rev, chosen_id = evaluate_ground_truth(user, rec_ids)

    results.append({
        "User ID": user['user_id'],
        "Profile": user['profile_name'],
        "Rec IDs": rec_ids,
        "Rec Names": rec_names,
        "Chosen ID": chosen_id,
        "Chosen Name": rest_name_map.get(chosen_id, "None"),
        "Revenue": rev
    })

ml_report_df = pd.DataFrame(results)

print("\n" + "="*40)
print(f"ML PIPELINE RESULTS")
print("="*40)
print(f"Total Revenue:   ${ml_report_df['Revenue'].sum():.2f}")
print(f"Avg Rev / User:  ${ml_report_df['Revenue'].mean():.2f}")
print("="*40)

# Display sample
pd.set_option('display.max_colwidth', None)
display(ml_report_df.head(5))

--- 1. Loading Data ---
Loaded 100 restaurants.
Loaded 200 transactions.

--- 2. Preparing ML Training Data ---
Training Data: 1000 rows.
Class Balance: 20.0% positive (chosen).

--- 3. Training Random Forest ---
Feature Importances:
  CuisineMatch: 0.3824
  PriceGap    : 0.2196
  ETA         : 0.2189
  Rating      : 0.1422
  PriceRaw    : 0.0368

--- 4. Defining ML Optimizer ---

--- 5. Defining Ground Truth (Oracle) ---

--- 6. Running ML Pipeline Evaluation (N=200) ---

ML PIPELINE RESULTS
Total Revenue:   $399.00
Avg Rev / User:  $2.00


,User ID,Profile,Rec IDs,Rec Names,Chosen ID,Chosen Name,Revenue
0,0,Budget Shopper,"[71, 9, 52, 64, 33]","[Farmhouse Kitchen Thai Cuisine, Skates On The Bay, Daryoush, Heroic Italian, Bangkok Noodles & Thai BBQ]",71,Farmhouse Kitchen Thai Cuisine,3
1,1,Budget Shopper,"[66, 2, 64, 33, 56]","[Ike's Love & Sandwiches, Comal, Heroic Italian, Bangkok Noodles & Thai BBQ, East Bay Spice Company]",66,Ike's Love & Sandwiches,2
2,2,Quality-Driven Healthy Eater,"[71, 20, 66, 19, 17]","[Farmhouse Kitchen Thai Cuisine, Triple Rock Brewery, Ike's Love & Sandwiches, Jupiter, Angeline's Louisiana Kitchen]",20,Triple Rock Brewery,2
3,3,Balanced Generalist,"[13, 61, 19, 20, 52]","[The Butcher's Son, Gadani, Jupiter, Triple Rock Brewery, Daryoush]",13,The Butcher's Son,2
4,4,Convenience Seeker,"[59, 35, 52, 61, 17]","[Eureka!, Bench Cafe Patisserie, Daryoush, Gadani, Angeline's Louisiana Kitchen]",59,Eureka!,2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import folium

# =============================================================================
# 7. ML VISUALIZATION (FOLIUM)
# =============================================================================
print("--- 7. Visualizing ML Assortments ---")

def visualize_ml_choices(report_df, users_df, restaurants_df, num_samples=5):
    # 1. Setup Map (Simple 0-10 Grid)
    m = folium.Map(
        location=[5, 5],
        zoom_start=4,
        crs="Simple",
        tiles=None
    )

    # White background
    folium.TileLayer(
        tiles="https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/blank.png",
        attr="Blank", name="Blank Background"
    ).add_to(m)

    # 2. Sample random users
    sample_rows = report_df.sample(n=min(num_samples, len(report_df)))

    # Helper lookups
    rest_lookup = restaurants_df.set_index('restaurant_id')[['x', 'y', 'name', 'cuisine']].to_dict('index')
    user_lookup = users_df.set_index('user_id')[['x', 'y']].to_dict('index')

    # 3. Plot Loop
    for _, row in sample_rows.iterrows():
        u_id = row['User ID']
        rec_ids = row['Rec IDs']     # <--- Adjusted for ML DataFrame
        chosen_id = row['Chosen ID']

        # Get User Coords
        u_data = user_lookup.get(u_id)
        if not u_data: continue

        # MARKER: User
        folium.Marker(
            location=[u_data['y'], u_data['x']],
            popup=f"User {u_id}<br>Profile: {row['Profile']}", # <--- Adjusted for ML DataFrame
            icon=folium.Icon(color='purple', icon='user', prefix='fa') # Purple for ML
        ).add_to(m)

        # LINES: Connect User to Recommended Restaurants
        for rid in rec_ids:
            r_data = rest_lookup.get(rid)
            if not r_data: continue

            # Style: Green/Thick if Chosen, Grey/Thin if just Recommended
            is_chosen = (rid == chosen_id)
            color = 'green' if is_chosen else 'gray'
            weight = 5 if is_chosen else 2
            opacity = 0.9 if is_chosen else 0.4

            # Draw Line
            folium.PolyLine(
                locations=[[u_data['y'], u_data['x']], [r_data['y'], r_data['x']]],
                color=color,
                weight=weight,
                opacity=opacity
            ).add_to(m)

            # MARKER: Restaurant
            folium.CircleMarker(
                location=[r_data['y'], r_data['x']],
                radius=6,
                color=color,
                fill=True,
                fill_opacity=0.7,
                popup=f"<b>{r_data['name']}</b> ({r_data['cuisine']})<br>{'✅ CHOSEN' if is_chosen else 'Rec (ML)'}"
            ).add_to(m)

    return m

# Generate the map
ml_viz_map = visualize_ml_choices(ml_report_df, users, restaurants, num_samples=5)
ml_viz_map

--- 7. Visualizing ML Assortments ---


In [ ]:
# =============================================================================
# ML HIT RATE CALCULATION (Global Best)
# =============================================================================
import numpy as np

def calculate_ml_hit_rate(report_df, users_df, restaurants_df):
    hits = 0
    total = len(report_df)

    print(f"Evaluating ML Hit Rate for {total} users...")

    for _, row in report_df.iterrows():
        u_id = row['User ID']

        # 1. Get the list of 5 IDs ML model recommended
        rec_ids = row['Rec IDs']

        # 2. Find the TRUE Global Best restaurant for this user
        user = users_df[users_df['user_id'] == u_id].iloc[0]

        best_util = -np.inf
        global_best_id = -1

        for _, rest in restaurants_df.iterrows():
            # Use the utility function from ML pipeline
            u = get_gt_utility(user, rest)

            if u > best_util:
                best_util = u
                global_best_id = rest['restaurant_id']

        # 3. Check if the Global Best is in Recommendation
        if global_best_id in rec_ids:
            hits += 1

    return hits / total

# Run Calculation
ml_hit_rate = calculate_ml_hit_rate(ml_report_df, users, restaurants)
print(f"\n>>> ML Global Hit Rate: {ml_hit_rate:.1%}")

Evaluating ML Hit Rate for 200 users...

>>> ML Global Hit Rate: 63.0%


In [ ]:
# =============================================================================
# METRIC: AVERAGE USER UTILITY (HAPPINESS)
# =============================================================================
def calculate_avg_utility(report_df, users_df, restaurants_df):
    total_util = 0
    count = 0

    for _, row in report_df.iterrows():
        u_id = row['User ID'] if 'User ID' in row else row['user_id']

        # Get the ID of the item the user ACTUALLY chose
        chosen_id = row['Chosen ID']

        user = users_df[users_df['user_id'] == u_id].iloc[0]
        rest = restaurants_df[restaurants_df['restaurant_id'] == chosen_id].iloc[0]

        # Calculate True Utility
        u = get_gt_utility(user, rest)
        total_util += u
        count += 1

    return total_util / count

#mnl_util = calculate_avg_utility(report_df, users, restaurants)
ml_util = calculate_avg_utility(ml_report_df, users, restaurants)

#print(f"MNL Avg User Utility: {mnl_util:.2f}")
print(f"ML  Avg User Utility: {ml_util:.2f}")

ML  Avg User Utility: 1.34


Extension 1: The "Profile Scorecard"
This code groups our results by the 10 profiles (e.g., "Speed Obsessed", "Budget Shopper") to see who we are serving well and who we are ignoring.

What to look for:

MNL: Is it making all its money from "Price Insensitive" profiles?"
ML: Is it failing specifically on "Complex" profiles like "Balanced Generalist"?

In [ ]:
# =============================================================================
# EXTENSION 1: BREAKDOWN BY PROFILE
# =============================================================================
def get_profile_breakdown(report_df, model_name):
    """
    Aggregates Revenue and Average Price Chosen by User Profile.
    """
    # 1. Group by Profile
    col = 'Profile Name' if 'Profile Name' in report_df.columns else 'Profile'

    breakdown = report_df.groupby(col).agg(
        User_Count=('User ID', 'count'),
        Total_Revenue=('Revenue', 'sum'),
        Avg_Revenue=('Revenue', 'mean'),
        # Calculate how often they picked the most expensive tier (Price=3)
        Premium_Pick_Rate=('Revenue', lambda x: (x==3).mean())
    ).sort_values("Total_Revenue", ascending=False)

    breakdown['Model'] = model_name
    return breakdown

# Run for ML
ml_profile_stats = get_profile_breakdown(ml_report_df, "ML") # Uses ML report_df
print("\n--- ML Performance by Profile ---")
display(ml_profile_stats)


--- ML Performance by Profile ---


,User_Count,Total_Revenue,Avg_Revenue,Premium_Pick_Rate,Model
Profile,,,,,
Budget Shopper,26,59,2.269231,0.423077,ML
Speed-Obsessed,32,59,1.843750,0.031250,ML
Rating Snob,27,54,2.000000,0.148148,ML
Balanced Generalist,23,47,2.043478,0.260870,ML
Cuisine-Focused Foodie,21,39,1.857143,0.142857,ML
Quality-Driven Healthy Eater,18,35,1.944444,0.111111,ML
Rating + Speed Hybrid,15,31,2.066667,0.066667,ML
Curious Food Explorer,16,31,1.937500,0.062500,ML
Slow but Cheap,11,24,2.181818,0.272727,ML


Extension 2: The "Missed Opportunity" Analysis
It calculates "Revenue Gap": The difference between the revenue of the user's absolute favorite item and the revenue of what they actually chose from our options.

- Gap = $0: Perfect extraction (or they prefer cheap food).

- Gap > $0: We left money on the table (e.g., their favorite was a $3 item, but our menu forced them to pick a $2 item).

- Gap < $0: The Upsell! (Their favorite was $1, but we successfully got them to buy a $2 item). This is where MNL shines.

In [ ]:
# =============================================================================
# EXTENSION 2: REVENUE GAP ANALYSIS
# =============================================================================
def analyze_revenue_gap(report_df, users_df, restaurants_df):
    gap_data = []

    for _, row in report_df.iterrows():
        u_id = row['User ID'] if 'User ID' in row else row['user_id']
        actual_rev = row['Revenue']

        # Find True Favorite Price
        user = users_df[users_df['user_id'] == u_id].iloc[0]
        best_util = -np.inf
        fav_price = 0

        for _, rest in restaurants_df.iterrows():
            # (Replace get_gt_utility with get_gt_features_and_utility if needed)
            u = get_gt_utility(user, rest)
            if u > best_util:
                best_util = u
                fav_price = rest['price_level']

        # Calculate Gap: (Actual - Favorite)
        # Positive = You upsold them!
        # Negative = You undersold them (or they settled for less)
        gap_data.append(actual_rev - fav_price)

    return pd.Series(gap_data)

# Calculate Gaps
ml_gaps = analyze_revenue_gap(ml_report_df, users, restaurants)

print(f"ML  Avg Upsell: ${ml_gaps.mean():.2f}")

# Visualize
import matplotlib.pyplot as plt
plt.figure(figsize=(10,5))
plt.hist(ml_gaps, alpha=0.5, label='ML', bins=[-2.5, -1.5, -0.5, 0.5, 1.5, 2.5])
plt.legend()
plt.title("Revenue Gap Distribution (Right = Good Upsell)")
plt.xlabel("Actual Revenue minus Favorite Item Revenue")
plt.show()

Extension 3: price sensitivity

In [ ]:
# =============================================================================
# EXTENSION 3: PRICE SENSITIVITY MATRIX (ML VERSION)
# =============================================================================

# 1. Merge Tolerance into the ML report
uid_col = 'User ID' if 'User ID' in ml_report_df.columns else 'user_id'

merged_ml = ml_report_df.merge(users[['user_id', 'price_tolerance']], left_on=uid_col, right_on='user_id')

print("--- ML: User Tolerance vs Chosen Price ---")
# 2. Create the Cross-Tabulation
matrix_ml = pd.crosstab(
    merged_ml['price_tolerance'],
    merged_ml['Revenue'],
    rownames=['User Tolerance'],
    colnames=['Chosen Price']
)

display(matrix_ml)

# 3. Calculate "Upsell Rate" (Percentage of users buying ABOVE their tolerance)
# Note: This is rare because penalty is high, but interesting to check.
upsells = merged_ml[merged_ml['Revenue'] > merged_ml['price_tolerance']].shape[0]
print(f"\nTotal users forced to buy ABOVE tolerance: {upsells}")

In [ ]:
# =============================================================================
# 3. TRAIN XGBOOST (WITH SIMPLE HYPERPARAMETER TUNING)
# =============================================================================
print("\n--- 3. Training XGBoost (with tuning) ---")

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# Train/validation split so we can see which params actually work better
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train,
    test_size=0.2,
    random_state=42,
    stratify=y_train
)

# Handle class imbalance via scale_pos_weight (computed on training split)
pos = (y_tr == 1).sum()
neg = (y_tr == 0).sum()
scale_pos_weight = neg / pos if pos > 0 else 1.0
print(f"  Class imbalance (train): pos={pos}, neg={neg}, scale_pos_weight={scale_pos_weight:.2f}")

# A small hyperparameter grid to search over
param_grid = [
    {"max_depth": 3, "n_estimators": 200, "learning_rate": 0.05, "subsample": 0.8, "colsample_bytree": 0.8},
    {"max_depth": 3, "n_estimators": 400, "learning_rate": 0.05, "subsample": 0.9, "colsample_bytree": 0.9},
    {"max_depth": 4, "n_estimators": 300, "learning_rate": 0.1,  "subsample": 0.8, "colsample_bytree": 0.8},
    {"max_depth": 5, "n_estimators": 300, "learning_rate": 0.05, "subsample": 0.8, "colsample_bytree": 0.8},
]

best_auc = -np.inf
best_model = None
best_params = None

for i, params in enumerate(param_grid, start=1):
    print(f"\n  >> Training config {i}/{len(param_grid)}: {params}")

    model = XGBClassifier(
        objective="binary:logistic",
        eval_metric="logloss",
        scale_pos_weight=scale_pos_weight,
        random_state=42,
        n_jobs=-1,
        **params
    )

    model.fit(X_tr, y_tr)

    # Evaluate on validation set (AUC is good for imbalanced problems)
    val_proba = model.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, val_proba)
    print(f"     Validation AUC: {auc:.4f}")

    if auc > best_auc:
        best_auc = auc
        best_model = model
        best_params = params

print("\nBest XGBoost params found:")
print(best_params)
print(f"Best validation AUC: {best_auc:.4f}")

# Use the best tuned model going forward
xgb_model = best_model

# Feature Importance Check
importances = dict(zip(feature_names, xgb_model.feature_importances_))
print("\nFeature Importances (from tuned XGBoost):")
for f, imp in sorted(importances.items(), key=lambda x: x[1], reverse=True):
    print(f"  {f:12s}: {imp:.4f}")

# =============================================================================
# 6. RUNNING EVALUATION & REPORTING
# =============================================================================
print("\n--- 6. Running ML Pipeline Evaluation (N=200) ---")

rest_name_map = restaurants.set_index('restaurant_id')['name'].to_dict()
results = []
test_users = users.head(200)

for _, user in test_users.iterrows():
    # 1. Optimize (ML using XGBoost)
    rec_ids = get_ml_optimal_assortment(user, xgb_model, K=5)
    rec_names = [rest_name_map[rid] for rid in rec_ids]

    # 2. Evaluate (Oracle)
    rev, chosen_id = evaluate_ground_truth(user, rec_ids)

    results.append({
        "User ID": user['user_id'],
        "Profile": user['profile_name'],
        "Rec IDs": rec_ids,
        "Rec Names": rec_names,
        "Chosen ID": chosen_id,
        "Chosen Name": rest_name_map.get(chosen_id, "None"),
        "Revenue": rev
    })

ml_report_df = pd.DataFrame(results)

print("\n" + "="*40)
print(f"ML PIPELINE RESULTS (XGBoost)")
print("="*40)
print(f"Total Revenue:   ${ml_report_df['Revenue'].sum():.2f}")
print(f"Avg Rev / User:  ${ml_report_df['Revenue'].mean():.2f}")
print("="*40)

# Display sample
pd.set_option('display.max_colwidth', None)
display(ml_report_df.head(5))
